In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import mne

In [ ]:
# some viz config
small_size, medium_size, bigger_size = 13, 18, 18

plt.rc('font', size=small_size)          # controls default text sizes
plt.rc('axes', titlesize=small_size)     # fontsize of the axes title
plt.rc('axes', labelsize=medium_size)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('ytick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('legend', fontsize=small_size)    # legend fontsize
plt.rc('figure', titlesize=bigger_size)  # fontsize of the figure title

## Let's start all over again

We will just read in the data as we did before, this time focusing on covariance matrices which, when combined with more advanced feature modeling, may improve performance

In [ ]:
from sklearn.model_selection import train_test_split


df_demographics = pd.read_csv('./inputs/Demographic_data.csv', header=1)

# remove empty columns
df_demographics = df_demographics.iloc[:, :5].set_index('Code')
df_demographics


# The later code uses the prefix "sub-" in the participants identifier, we will add it here to be fine

df_demographics.index = "sub-" + df_demographics.index
df_demographics.index

In [ ]:
# now we read in the processing log to see for which participants we have EEG

proc_log = pd.read_csv('./outputs/autoreject_log.csv')
good_subjects = proc_log.query('ok == "OK"').subject
good_subjects

In [ ]:
# then we filter the demographic list accordingly and establish the same order

df_demographics = df_demographics.loc[good_subjects]

In [ ]:
# Now we can put some data aside for testing and focus on 80 percent of the cases for exploring

train_cases, test_cases = train_test_split(df_demographics, test_size=.20, random_state=42)

## Read the pre-computed features

We first start with the power spectra. As we read, we make sure features are stored in
the same order as our meta info.

In [ ]:
features = mne.externals.h5io.read_hdf5('./outputs/features_eyes-closed.h5')
covs = [features[sub]['covs'] for sub in train_cases.index]
X_covs = np.array(covs)
print(X_covs.shape)  # hooray we have an array! :D

In [ ]:
# But what are these dimensions?

## Let's explore these covariances

In [ ]:
train_cases['age_group'] = pd.cut(train_cases.Age, 4)

In [ ]:
# now we can do a group by after setting the index to range
train_cases = train_cases.reset_index()


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

color = plt.cm.viridis(np.linspace(0.1, 0.9, 4))
fig, axes = plt.subplots(1, 4, figsize=(14, 9))
for ii, (key, inds) in enumerate(train_cases.groupby('age_group').groups.items()):
    im = axes[ii].matshow(X_covs[inds][1].mean(0), cmap='RdBu')
    axes[ii].set_title(f'Age in {key}')
    divider = make_axes_locatable(axes[ii])
    cax = divider.append_axes('right', size='5%', pad=0.2)
    fig.colorbar(im, cax=cax)

fig.tight_layout()

In [ ]:
ch_counts = pd.read_csv('./outputs/channel_counts.csv')

ch_counts

What we see here suggests there is highly structured noise. This may be atypical. **If you see something like this please go back to your preprocessing work and and double check that all is as expected.**

# A more advanced model

Shall we still try to see if a more sophisticated model can do the job? Let's pretend we have not seen anything.

Now the idea is to first put all thes covariances in a data frame where things become easier to handle.

In [ ]:
frequency_bands = {
    "theta": (4.0, 8.0),
    "alpha": (8.0, 15.0),
    "beta_low": (15.0, 26.0),
    "beta_high": (26.0, 35.0),
}

X_df = pd.DataFrame(
  {band: list(X_covs[:, ii]) for ii, band in enumerate(frequency_bands)})

In [ ]:
X_df.shape

In [ ]:
X_df.columns

In [ ]:
X_df

Covariances in a Data Frame!

In [ ]:
 y = train_cases.Age.values

In [ ]:
from sklearn.linear_model import RidgeCV, GammaRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import coffeine

In [ ]:
filter_bank_transformer = coffeine.make_filter_bank_transformer(
    names=['theta', 'alpha', 'beta_low', 'beta_high'], method='riemann',
    projection_params=dict(n_compo=40))


filter_bank_model = make_pipeline(filter_bank_transformer, StandardScaler(),
                                  RidgeCV(alphas=np.logspace(-3, 10, 100)))

y_pred = cross_val_predict(estimator=filter_bank_model, X=X_df, y=y)
plt.scatter(y, y_pred)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_true=y, y_pred=y_pred))

From a theoretical standpoint and empirically, this type of model is expected to work better. We can think a moment together about it. It is most likely that something is seriously wrong about the way we are using and processing the data, which can be very tricky when first encountering a new curated dataset. Something to be clarified in the nearer future; **But we're working on it together with the CHBM team**!

But it does not mean we have to stop here. The coarse patterning in the covariance matrix susggest a dominant low-rank structure. We can explore that further.

In [ ]:
from scipy.linalg import svd
s_values = np.sqrt(np.array([svd(cc[1], compute_uv=False) for cc in X_covs]))

In [ ]:
plt.plot(s_values.T);

We now see that a few directions of variance dominate the spectrum. And one subject seems to even have flat spectrum. **Is it the same subject that we exculded prviously**? And does it matter?

This together with the bad result obtained with the Riemannian embedding suggests that there is some model violation. We should explore more optinos in coffeine's filter bank model